In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import xgboost as xgb
import catboost as catb

from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import MinMaxScaler
from tkinter.tix import COLUMN
from sklearn.model_selection import TimeSeriesSplit

pd.options.mode.chained_assignment = None

SCALER = MinMaxScaler()

In [2]:
df = pd.read_csv("data.csv").rename(columns={'England\n[note 2]' : 'England'}).drop(columns = ["Weekday", "Total"])
pricesDF = pd.read_csv("prices.csv").set_index("Date")
curfewDF = pd.read_csv("curfew.csv").set_index("Date")
for ind in df.index:
    date = df['Date'][ind]
    date = str(date)

    date1 = date.replace("T", " ")
    date2 = date1.replace("Z", "")

    df['Date'][ind] = date2

df = df.set_index("Date")

Diesel_DF = df.loc[df["Fuel Type"] == "Diesel"].drop(columns= {"Fuel Type"})
Petrol_DF = df.loc[df["Fuel Type"] == "Petrol"].drop(columns= {"Fuel Type"})

Diesel_DF.index = pd.to_datetime(Diesel_DF.index)
Petrol_DF.index = pd.to_datetime(Petrol_DF.index)

REGIONS = Diesel_DF.columns.values[:6]

# total = []

# for index, row in Diesel_DF.iterrows():
#     value = 0

#     for idxCol in range(len(Diesel_DF.columns)):
#         value += row.values[idxCol]

#     total.append(value)

# Diesel_DF["Total"] = total

# Diesel_DF
# total = []

# for index, row in Petrol_DF.iterrows():
#     value = 0

#     for idxCol in range(len(Petrol_DF.columns)):
#         value += row.values[idxCol]

#     total.append(value)

# Petrol_DF["Total"] = total


In [3]:
dieselPrices = pricesDF["Diesel Price"].values
# petrolPrices = pricesDF["Petrol Price"].values
curfewValues = curfewDF["Curfew"].values
Diesel_DF["price"] = dieselPrices
# Petrol_DF["price"] = petrolPrices
Diesel_DF["curfew"] = curfewValues
# Petrol_DF["curfew"] = curfewValues
# Diesel_DF.plot(figsize=(40,5))

Diesel_DF


,North East,North West,Yorkshire and The Humber,East Midlands,West Midlands,East,London,South East,South West,Wales,Scotland,England,price,curfew
Date,,,,,,,,,,,,,,
2020-01-27,10581.0,10827.0,12687.0,12897.0,11910.0,12221.0,8290.0,11143.0,11141.0,11176.0,10967.0,11339.0,132.88,0
2020-01-28,10420.0,10856.0,12658.0,13230.0,12388.0,12959.0,8433.0,11616.0,11496.0,11278.0,10975.0,11644.0,132.88,0
2020-01-29,10441.0,10788.0,12799.0,13242.0,12045.0,12705.0,8458.0,11530.0,11395.0,11127.0,10932.0,11557.0,132.88,0
2020-01-30,10501.0,10801.0,12596.0,12947.0,11936.0,12545.0,8209.0,11369.0,11302.0,11116.0,11553.0,11422.0,132.88,0
2020-01-31,11513.0,11065.0,13003.0,13560.0,12210.0,13105.0,8629.0,11946.0,12066.0,11863.0,11879.0,11923.0,132.88,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-07-20,9054.0,9349.0,10787.0,11421.0,10362.0,11282.0,6521.0,10153.0,10168.0,8553.0,9117.0,9995.0,197.53,0
2022-07-21,9270.0,9349.0,10691.0,11456.0,10568.0,11519.0,6566.0,10443.0,10491.0,8850.0,9412.0,10143.0,197.53,0
2022-07-22,9767.0,9288.0,10965.0,11778.0,10521.0,11516.0,6705.0,10386.0,10811.0,9189.0,9502.0,10249.0,197.53,0


In [4]:
# fig, axs = plt.subplots(len(REGIONS), 1, figsize=(40,30))

# for x in range(len(REGIONS)):
#     dfr = Diesel_DF[REGIONS[x]]
    
#     dfr.loc[dfr.index < "2022-04-27"].plot(ax=axs[x], title=f"{REGIONS[x]}")
#     dfr.loc[dfr.index > "2022-04-27"].plot(ax=axs[x])
#   fig.savefig("asd.pdf")

In [5]:
# Removal of outliers
for region in REGIONS:
    Diesel_DF.loc[Diesel_DF[region] > 15000] = Diesel_DF[region].mean()

In [ ]:
DEPTH = 3
ESTIMATORS = 500
CV_K = 5
TEST_SIZE = 90

def createFeatures(df):
    df = df.copy()

    df['dayofweek'] = df.index.dayofweek
    df['quarter'] = df.index.quarter
    df['month'] = df.index.month
    df['year'] = df.index.year
    df['dayofyear'] = df.index.dayofyear
    df['dayofmonth'] = df.index.day
    df['weekofyear'] = df.index.isocalendar().week.astype(int)
    return df

def addLags(df, regionName):
    target = df[regionName].to_dict()
    df["lag 1"] = (df.index - pd.Timedelta('1 days')).map(target)
    df["lag 2"] = (df.index - pd.Timedelta('2 days')).map(target)
    df["lag 3"] = (df.index - pd.Timedelta('3 days')).map(target)
    df["lag 4"] = (df.index - pd.Timedelta('4 days')).map(target)
    df["lag 5"] = (df.index - pd.Timedelta('5 days')).map(target)
    df["lag 6"] = (df.index - pd.Timedelta('6 days')).map(target)

    return df

def run(df, regionName, count, fig, axs):
    print(regionName)

    tss = TimeSeriesSplit(n_splits=CV_K, test_size=TEST_SIZE)

    xg = xgb.XGBRegressor(base_score=0.5, booster='gbtree',    
                       n_estimators=ESTIMATORS,
                       early_stopping_rounds=50,
                       objective='reg:linear',
                       max_depth=DEPTH,
                       learning_rate=0.01)

    cat = catb.CatBoostRegressor(early_stopping_rounds=50, 
                            max_depth=DEPTH,
                            n_estimators=ESTIMATORS,
                            learning_rate=0.01)

    fold = 0
    predsXGB = []
    predsCAT = []
    scoresXGB = []
    scoresCAT = []

    for train_idx, val_idx in tss.split(df):
        train = df.iloc[train_idx]
        test = df.iloc[val_idx]

        train = addLags(train, regionName)
        test = addLags(test, regionName)

        FEATURES = [
                    # "dayofyear", 
                    "dayofweek", 
                    # "quarter",
                    # "month",
                    "year", 
                    "price", 
                    "curfew", 
                    "lag 1",
                    "lag 2", 
                    "lag 3",
                    "lag 4"
                    # "lag 5",
                    # "lag 6",
                    # "weekofyear",
                    # "dayofmonth"
                    ]

        TARGET = [regionName]

        xTrain = train[FEATURES]
        yTrain = train[TARGET]

        xTest = test[FEATURES]
        yTest = test[TARGET]

        # Normalizing

        column = SCALER.fit_transform(xTrain[["price"]])
        xTrain["price"] = column
        column = SCALER.fit_transform(xTest[["price"]])
        xTest["price"] = column

        # XGB fit

        xg.fit(xTrain, yTrain,
            eval_set=[(xTrain, yTrain), (xTest, yTest)],
            verbose=100)

        yTest["prediction_xgb"] = xg.predict(xTest)
        prediction = yTest["prediction_xgb"]
        predsXGB.append(prediction)

        score = np.sqrt(mean_squared_error(yTest["prediction_xgb"], yTest[regionName]))
        scoresXGB.append(score)

        # CAT fit

        cat.fit(xTrain, yTrain, verbose=100)

        yTest["prediction_cat"] = cat.predict(xTest)
        prediction = yTest["prediction_cat"]
        predsCAT.append(prediction)

        score = np.sqrt(mean_squared_error(yTest["prediction_cat"], yTest[regionName]))
        scoresCAT.append(score)

        # Plotting

        # if fold == 4:
        yTest[regionName].plot(ax=axs[count + fold],
                                    label='Fuel Sales',
                                    title=f'{regionName} Data Train/Test Split Fold {fold + 1}')
        yTest["prediction_xgb"].plot(ax=axs[count + fold],
                                    label='XGB Predictions')
        yTest["prediction_cat"].plot(ax=axs[count + fold],
                                    label='Cat Predictions').legend()
        

        # Importances

        # imp = pd.DataFrame(data=xg.feature_importances_,
        #         index=xg.feature_names_in_,
        #         columns=['importance'])
        # imp.sort_values('importance').plot(ax=axs[fold + CV_K], kind='barh', title=f'{regionName} Feature Importance Fold {fold + 1}')

        fold += 1
    
    return scoresXGB[CV_K - 1], scoresCAT[CV_K - 1], fig, predsXGB[CV_K - 1], predsCAT[CV_K - 1]
    # return np.mean(scoresXGB), np.mean(scoresCAT), fig

In [ ]:
scoresXGB = []
scoresCAT = []
predsXGB = []
predsCAT = []
figs= []
fig, axs = plt.subplots(len(REGIONS) * CV_K, 1, figsize=(40,40))
count = 0

for region in REGIONS:
    regionDF = Diesel_DF[[region, "price", "curfew"]]                                                                 ##### RENAME COLUMNS #####
    regionDF = createFeatures(regionDF)

    scoreXGB, scoreCAT, fig, predXGB, predCAT = run(regionDF, region, count, fig, axs)

    scoresXGB.append(scoreXGB)
    scoresCAT.append(scoreCAT)
    predsXGB.append(predXGB)
    predsCAT.append(predCAT)
    figs.append(fig)

    Diesel_DF["XGB Pred"] = predXGB
    Diesel_DF["Cat Pred"] = predCAT

    count += 5

Diesel_DF

# fig.tight_layout(pad=0.5)
# fig.savefig(f"resultsRegi.pdf")

# totalDF = Diesel_DF[["Total", "price", "curfew"]]
# totalDF = createFeatures(totalDF)

# print(totalDF)


In [ ]:
# values = []
# for x in range(90):
#     value = 0
#     for i in range(len(REGIONS)):
#         value += predsXGB[i][x]
#     values.append(value)

# print(values)

In [ ]:
# fig, axs = plt.subplots(CV_K, 1, figsize=(40,40))

# count = 0
# totalDF = Diesel_DF[["Total", "price", "curfew"]]
# totalDF = createFeatures(totalDF)
# scoreXGB, scoreCAT, fig, totalPredXGB, totalPredCAT = run(totalDF, "Total", count, fig, axs)

# print(totalDF)
# regionsPredXGB = 0
# regionsPredCAT = 0

# for x in range(len(regionsPredXGB)):
#     totalPredXGB += predsXGB[x]

# print(regionsPredXGB)

In [ ]:
for idx in range(len(REGIONS)):
    print(f"Score for XGB in {REGIONS[idx]}: {scoresXGB[idx]}")
    print(f"Score for CAT in {REGIONS[idx]}: {scoresCAT[idx]}")

In [ ]:
# for x in range(len(figs)):
#     figs[x].savefig(f"{DEPTH}_{ESTIMATORS}_{CV_K}_{REGIONS[x]}.pdf")

In [ ]:
X = REGIONS
Ygirls = scoresXGB
Zboys = scoresCAT
  
X_axis = np.arange(len(X))

f, ax = plt.subplots(figsize=(20,20))
  
plt.bar(X_axis - 0.2, Ygirls, 0.4, label = 'XGB')
plt.bar(X_axis + 0.2, Zboys, 0.4, label = 'CAT')
  
plt.xticks(X_axis, X)
plt.xlabel("Regions")
plt.ylabel("Number of Students")
plt.title("Scores by each region")
plt.legend()
plt.show()

f.savefig("6d.pdf")

BELOW DEPRECEATED

In [ ]:
# tss = TimeSeriesSplit(n_splits=CV_K, test_size=90)

# xg = xgb.XGBRegressor(base_score=0.5, booster='gbtree',    
#                        n_estimators=ESTIMATORS,
#                        early_stopping_rounds=50,
#                        objective='reg:linear',
#                        max_depth=DEPTH,
#                        learning_rate=0.01)

# cat = catb.CatBoostRegressor(early_stopping_rounds=50, 
#                             max_depth=DEPTH,
#                             n_estimators=ESTIMATORS,
#                             learning_rate=0.01)

In [ ]:
# fig, axs = plt.subplots(CV_K * 2, 1, figsize=(40,40))

# fold = 0
# predsXGB = []
# predsCat = []
# scoresXGB = []
# scoresCat = []

# for train_idx, val_idx in tss.split(NE_diesel):
#     NE_diesel = createFeatures(NE_diesel)
#     train = NE_diesel.iloc[train_idx]
#     test = NE_diesel.iloc[val_idx]

#     FEATURES = ["dayofyear", "dayofweek", "quarter", "month", "year", "price", "curfew"]
#     TARGET = ["North East"]

#     xTrain = train[FEATURES]
#     yTrain = train[TARGET]

#     xTest = test[FEATURES]
#     yTest = test[TARGET]

#     # XGB fit

#     xg.fit(xTrain, yTrain,
#         eval_set=[(xTrain, yTrain), (xTest, yTest)],
#         verbose=100)
    
#     yTest["prediction_xgb"] = xg.predict(xTest)
#     prediction = yTest["prediction_xgb"]
#     predsXGB.append(prediction)

#     score = np.sqrt(mean_squared_error(yTest["prediction_xgb"], yTest["North East"]))
#     scoresXGB.append(score)

#     # Cat fit

#     cat.fit(xTrain, yTrain, verbose=100)

#     yTest["prediction_cat"] = cat.predict(xTest)
#     prediction = yTest["prediction_cat"]
#     predsCat.append(prediction)

#     score = np.sqrt(mean_squared_error(yTest["prediction_cat"], yTest["North East"]))
#     scoresCat.append(score)

#     # Plotting

#     yTest["North East"].plot(ax=axs[fold * 2],
#                             label='Fuel Sales',
#                             title=f'Data Train/Test Split Fold {fold}')
#     yTest["prediction_xgb"].plot(ax=axs[fold * 2],
#                             label='XGB Predictions',
#                             title=f'Data Train/Test Split Fold {fold}')
#     yTest["prediction_cat"].plot(ax=axs[fold * 2],
#                             label='Cat Predictions',
#                             title=f'Data Train/Test Split Fold {fold}').legend()

#     # Importances

#     imp = pd.DataFrame(data=xg.feature_importances_,
#              index=xg.feature_names_in_,
#              columns=['importance'])
#     imp.sort_values('importance').plot(ax=axs[fold * 2 + 1], kind='barh', title='Feature Importance')

#     fold += 1


In [ ]:
# print(f"Scores across XGB folds: {scoresXGB}")
# print(f"Mean of XGB cores: {np.mean(scoresXGB):0.4f}\n")
# print(f"Scores across CAT folds: {scoresCat}")
# print(f"Mean of CAT scores: {np.mean(scoresCat):0.4f}\n")

In [ ]:
# def splitData(df):
#     train = df.loc[df.index < '01-27-2022']
#     test = df.loc[df.index >= '01-27-2022']

#     fig, ax = plt.subplots(figsize=(30, 5))
#     train[df.columns.values[0]].plot(ax=ax, label='Training Set', title=df.columns.values[0])
#     test[df.columns.values[0]].plot(ax=ax, label='Test Set')
#     ax.axvline('01-27-2022', color='black', ls='--')
#     ax.legend(['Training Set', 'Test Set'])
#     plt.show()

#     return train, test

# def createFeatures(df):
#     """
#     Create time series features based on time series index.
#     """
#     df = df.copy()
#     # df['hour'] = df.index.hour
#     df['dayofweek'] = df.index.dayofweek
#     df['quarter'] = df.index.quarter
#     df['month'] = df.index.month
#     df['year'] = df.index.year
#     df['dayofyear'] = df.index.dayofyear
#     # df['dayofmonth'] = df.index.day
#     # df['weekofyear'] = df.index.isocalendar().week
#     return df

# NE_diesel_train, NE_diesel_test = splitData(createFeatures(NE_diesel))
# # NW_diesel_train, NW_diesel_test = splitData(createFeatures(NW_diesel))
# # Yorkshire_diesel_train, Yorkshire_diesel_test = splitData(createFeatures(Yorkshire_diesel))
# # EMid_diesel_train, EMid_diesel_test = splitData(createFeatures(EMid_diesel))
# # WMid_diesel_train, WMid_diesel_test = splitData(createFeatures(WMid_diesel))
# # East_diesel_train, East_diesel_test = splitData(createFeatures(East_diesel))
# # London_diesel_train, London_diesel_test = splitData(createFeatures(London_diesel))
# # SE_diesel_train, SE_diesel_test = splitData(createFeatures(SE_diesel))
# # SW_diesel_train, SW_diesel_test = splitData(createFeatures(SW_diesel))
# # Wales_diesel_train, Wales_diesel_test = splitData(createFeatures(Wales_diesel))
# # Scotland_diesel_train, Scotland_diesel_test = splitData(createFeatures(Scotland_diesel))
# # England_diesel_train, England_diesel_test = splitData(createFeatures(England_diesel))

In [ ]:
# FEATURES = ["dayofyear", "dayofweek", "quarter", "month", "year", "price", "curfew"]
# TARGET = NE_diesel_train.columns.values[0]

# xTrain = NE_diesel_train[FEATURES]
# yTrain = NE_diesel_train[TARGET]

# xTest = NE_diesel_test[FEATURES]
# yTest = NE_diesel_test[TARGET]

In [ ]:
# xg = xgb.XGBRegressor(base_score=0.5, booster='gbtree',    
#                        n_estimators=ESTIMATORS,
#                        early_stopping_rounds=50,
#                        objective='reg:linear',
#                        max_depth=DEPTH,
#                        learning_rate=0.01)
# xg.fit(xTrain, yTrain,
#         eval_set=[(xTrain, yTrain), (xTest, yTest)],
#         verbose=100)

In [ ]:
# fi_xg = pd.DataFrame(data=xg.feature_importances_,
#              index=xg.feature_names_in_,
#              columns=['importance'])
# fi_xg.sort_values('importance').plot(kind='barh', title='Feature Importance')
# plt.show()

In [ ]:
# cat = catb.CatBoostRegressor(early_stopping_rounds=50, 
#                             max_depth=DEPTH,
#                             n_estimators=ESTIMATORS,
#                             learning_rate=0.01)

# cat.fit(xTrain, yTrain, verbose=100)

In [ ]:
# fi_cat = pd.DataFrame(data=cat.feature_importances_,
#              index=cat.feature_names_,
#              columns=['importance'])
# fi_cat.sort_values('importance').plot(kind='barh', title='Feature Importance')
# plt.show()

In [ ]:
# NE_diesel_test["prediction_cat"] = cat.predict(xTest)
# NE_diesel_test['prediction_xgb'] = xg.predict(xTest)
# # print(NE_diesel_test)

# NE_diesel_predictions = NE_diesel_test.merge(NE_diesel_test[['prediction_xgb', "prediction_cat"]], how='left', left_index=True, right_index=True).drop(columns=["quarter", "month", "dayofweek", "year", "dayofyear"])

In [ ]:
# NE_diesel_predictions[["North East"]].loc[(NE_diesel_predictions.index >= "01-27-2022")].plot(figsize = (30,5), style= "", color = "black")
# NE_diesel_predictions.prediction_cat_x.plot(style= "", color = "orange")
# NE_diesel_predictions.prediction_xgb_x.plot(style= "", color = "red")
# plt.legend()
# plt.title('Raw Data and XGBoost Prediction')

In [ ]:
# XGBscore = np.sqrt(mean_squared_error(NE_diesel_predictions["North East"], NE_diesel_predictions["prediction_xgb_x"]))
# Catscore = np.sqrt(mean_squared_error(NE_diesel_predictions["North East"], NE_diesel_predictions["prediction_cat_x"]))
# print(f'XGBoost RMSE Score on Test set: {XGBscore:0.2f}')
# print(f'CatBoost RMSE Score on Test set: {Catscore:0.2f}')